In [2]:
"""!conda install -c conda-forge openexr-python -y
!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir 
!pip install --upgrade google-cloud-storage"""

'!conda install -c conda-forge openexr-python -y\n!pip install waymo-open-dataset-tf-2-11-0==1.5.2  --no-cache-dir \n!pip install --upgrade google-cloud-storage'

In [3]:
# Imports
import os
import tarfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm

from waymo_open_dataset.wdl_limited.sim_agents_metrics import metric_features
from waymo_open_dataset.wdl_limited.sim_agents_metrics import metrics

from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import sim_agents_metrics_pb2
from waymo_open_dataset.protos import sim_agents_submission_pb2
from google.protobuf import text_format

from waymo_open_dataset.utils.sim_agents import submission_specs
from waymo_open_dataset.utils.sim_agents import test_utils as sim_agents_test_utils
from waymo_open_dataset.utils.sim_agents import visualizations
from waymo_open_dataset.utils import trajectory_utils

# Set matplotlib to jshtml so animations work with colab.
from matplotlib import rc
rc('animation', html='jshtml')

from download import download_from_gcs


2023-05-26 15:29:09.530099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 15:29:13.087618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-26 15:29:13.087657: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-26 15:29:23.048083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

# Downloader

In [7]:
# Download samples
download_from_gcs('uncompressed/scenario/validation/validation.tfrecord-00000-of-00150')
download_from_gcs('uncompressed/scenario/training/training.tfrecord-00000-of-01000')
download_from_gcs('uncompressed/scenario/testing/testing.tfrecord-00000-of-00150')

FileNotFoundError: [Errno 2] No such file or directory: 'waymo-od-1-2-0-fb3a3a0b6e6e.json'

# Loader

In [4]:
DATASET_FOLDER = '/content/drive/MyDrive/Colab Notebooks/waymo_open_dataset_'

TRAIN_FILES = os.path.join(DATASET_FOLDER, 'training.tfrecord*')
VALIDATION_FILES = os.path.join(DATASET_FOLDER, 'validation.tfrecord*')
TEST_FILES = os.path.join(DATASET_FOLDER, 'testing.tfrecord*')

In [5]:
def Prepare_train_dataset():
    filenames = tf.io.matching_files(TRAIN_FILES)
    dataset = tf.data.TFRecordDataset(filenames)
    return dataset

def Prepare_validation_dataset():
    filenames = tf.io.matching_files(VALIDATION_FILES)
    dataset = tf.data.TFRecordDataset(filenames)
    return dataset

def Prepare_test_dataset():
    filenames = tf.io.matching_files(TEST_FILES)
    dataset = tf.data.TFRecordDataset(filenames)
    return dataset

In [6]:
# scenario exemple
dataset_iterator = Prepare_train_dataset().as_numpy_iterator()
bytes_example = next(dataset_iterator)
scenario = scenario_pb2.Scenario.FromString(bytes_example)
print(f'Checking type: {type(scenario)}')
print(f'Loaded scenario with ID: {scenario.scenario_id}')

Checking type: <class 'waymo_open_dataset.protos.scenario_pb2.Scenario'>
Loaded scenario with ID: 4b60f9400a30ceaf


In [11]:
# get the list of all the possible functions of the scenario object
print(f'List of all the fields of the scenario object: {[f for f in dir(scenario) if not callable(f)]}')

List of all the fields of the scenario object: ['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_SetListener', '__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', '_extensions_by_name', '_extensions_by_number', 'compressed_frame_laser_data', 'current_time_index', 'dynamic_map_states', 'm